In [1]:
import glob 
import os
import numpy as np
import matplotlib.pyplot as plt
import pydicom
import cv2
from scipy.signal import find_peaks
import plotly.graph_objects as go
import plotly.express as px
import torch

In [2]:
def plotly_visualization(x, y, title=''):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=y, mode='lines', name='ECG'))
    fig.add_trace(go.Scatter(x=x, y=y[x], mode='markers', marker=dict(symbol='x', size=8, color='red'), name='R-peaks'))

    fig_width = 800 
    fig_height = 400  
    fig.update_layout(
        title=title,
        yaxis= dict(autorange='reversed'),
        width=fig_width,
        height=fig_height
    )

    fig.show()

In [3]:
def img_visualization(image, title='', color=''):
    if color=='gray':
        fig = px.imshow(image, color_continuous_scale='gray')
    else:
        fig = px.imshow(image)
    fig.update_layout(
        title=title,
        width=800,
        height=200,
        coloraxis_showscale=False
    )
    fig.show()

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cpu


#### Checking for the color space (if YCbCr then color mapping is required)

In [5]:
def find_color_space(dcm):
    # Find through meta data
    transfer_syntax_uid = dcm.file_meta.TransferSyntaxUID
    if transfer_syntax_uid == ('1.2.840.10008.1.2.4.50' or '1.2.840.10008.1.2.4.51' or '1.2.840.10008.1.2.4.57' or '1.2.840.10008.1.2.4.70'):
        # print('YCbCr')
        return 'YCbCr'
    else:
        # print('RGB')
        return 'RGB'

##### Detecting R-peaks and returning their spatial information 

In [6]:
def detect_peaks(y_coordinates):
    # Detect R-peaks

    peaks = list()
    # last_non_none = next((item for item in reversed(y_coordinates) if item is not None), None)
    y_coordinates_non_none = y_coordinates[y_coordinates != None ] 
    # as its a negative number I want to keep the actual low so the negative threshold will be high
    threshold_value =  -int((y_coordinates_non_none.max() + y_coordinates_non_none.min())/2)
    # assigning a high value that doesn't affect peaks as find_peaks() input shouldn't have none vals
    y_coordinates[y_coordinates == None] = 1000

    #indicating fare x distance between each R peak based on the range of x
    xrange_ECG = np.where(y_coordinates!=1000)[0]
    x_distance = int((xrange_ECG[-1] - xrange_ECG[0])/4)
    
    x_peaks, _ = find_peaks(-y_coordinates, distance=x_distance)
    # threshold to filter out peaks below a certain amplitude
    x_peaks = [x for x in x_peaks if -y_coordinates[x] > threshold_value]

    y_coordinates[y_coordinates == 1000] = None

    plotly_visualization(x_peaks, y_coordinates, "ECG with Detected R-peaks")

    for x_peak in x_peaks:
        peaks.append((x_peak, y_coordinates[x_peak]))

    return peaks

##### Converting to 1D vector

In [7]:
def vector1D(image2D):
 
    # For each column in the image, find the y-coordinate of the curve
    y_coordinates = list()
    for x in range(image2D.shape[1]):
        column = image2D[:, x]
        indices = np.where(column > 0)
        if indices[0].size > 0:
            y_coordinate = int(np.min(indices))  # Max of the non-background pixels positions in the column
            y_coordinates.append(y_coordinate)
        else:
            y_coordinates.append(None)
  
    y_coordinates = np.array(y_coordinates)
    # print('-----------------',y_coordinates)
    return y_coordinates


In [28]:
def peak_extraction(mask):
    heights_with_line = []
    coordinates = []
    threshhold = 0
    for i in range(mask.shape[0]):
        if(mask[i].sum() > 120):
            heights_with_line.append(i)
    threshhold = (heights_with_line[0] - heights_with_line[-1]) /2


    # heights_with_line -= threshhold
    # filtered_high_heights = heights_with_line[heights_with_line>0]

    for i in range(mask.shape[0]):
        if(i> threshhold and len(coordinates)< 3 ):
            if (mask[i].sum()>120):
                potential_x = np.where(mask[i] > 120) 
                for x in potential_x[0]:
                    if (i > 0) and (mask[i-1, x] == 0) and (mask[i, x+10] == 0) and (mask[i, x-10] == 0):
                        coordinates.append((x, i))
                        break  
        else:
            break    

            
    return sorted(coordinates)

# mask = cv2.imread("d:/University/1-Calgary/experiments/1 - reading images/filtered_image.jpg")
# mask = cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY)
# print(peak_extraction(mask))
# img_visualization(mask)

In [9]:
def green_line_extractor(image):
   
   # if color_map == 'YCbCr':
   image = cv2.cvtColor(np.array(image), cv2.COLOR_YCrCb2RGB)

    # converting to hsv
   hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

   # Define the range for the green color
   lower_green = np.array([40, 40, 40])
   upper_green = np.array([80, 255, 255])

   # Create a mask for the green color
   mask = cv2.inRange(hsv_image, lower_green, upper_green)

   # Set non-green pixels to 0 and green pixels to a specific value
   # filtered_image = cv2.bitwise_and(image, image, mask=mask)

   return mask


In [10]:
def extraction(image):

    mask = green_line_extractor(image)
    img_visualization(mask, "Exteracted ECG line from image", 'gray')
    # cv2.imwrite(f'{name}.jpg', filtered_image)
    # vector = vector1D(mask)
    # coordinates = detect_peaks(vector)
    coordinates = peak_extraction(mask)
    return coordinates 


##### Extracting ECG for each cardiac slice

In [35]:
def cycle_indentifier(image, coordinates):
    # torch.tensor(coordinates, device=device)
    indices = list()
    slice = 0

    # for num, coordinate in enumerate(coordinates):
    #     x_axis = coordinate[0]
    #     expected_h = coordinate[1]
    expected_x = [i for i,_ in coordinates]
    expected_h = [j for _,j in coordinates]

    while(slice < image.shape[0]):
        test = image[slice, :, :, :]
        signal = torch.tensor(green_line_extractor(test), device = device)
        for i in range(signal.shape[0]):
            if (signal[i].sum() > 120) and (i in expected_h) and len(set(np.where(signal[i]>120)[0]).intersection(expected_x)) != 0:
                print(np.where(signal[i]>120)[0])
                print(f' cycle is completed at time index {slice}')
                indices.append(slice)
                slice += 5
            else:
                slice += 1
            #     heights.append(i)

            # if (heights):
            #     if (expected_h == int(min(heights))) or (expected_h in heights):
            #         print(f'{num+1} cycle is completed at time index {x}')
            #         # next_temporal = x
            #         indices.append(x)
            #         break

    return indices
                

In [29]:
images = list()
paths = glob.glob(os.path.join('d:/University/1-Calgary/experiments/1 - reading images/ii/', '*' ))
dcms = [pydicom.dcmread(path) for path in paths]
for dcm in dcms:
    img = torch.tensor(dcm.pixel_array, device=device)
    if len(img.shape)<4:
        continue
    offset = int(8/9*img.shape[1])
    images.append(img[:,offset:,:,:])

In [36]:
for i, image in enumerate(images):
    temporal_indecies = list()
    # color_map = find_color_space(dcm)

    print(i+1)
    img_visualization(image[-1,:,:,:])
    # pass the 3D image
    coordinates  = extraction(image[-1,:,:,:].squeeze())
    print(coordinates)
    
    
    temporal_indecies.append(cycle_indentifier(image, coordinates))

    # print('\n')
    # for temporal_index in temporal_indecies[0]:
    #     img_visualization(image[temporal_index,:,:,:])

    print('---------------------------------------------------------------')
 

1


[(68, 13), (163, 11), (164, 10)]
[164 165]
 cycle is completed at time index 70
[163 164 165]
 cycle is completed at time index 75
[ 68  69 162 163 164 165]
 cycle is completed at time index 81
---------------------------------------------------------------
2


[(65, 17), (156, 16), (158, 18)]
[65 66]
 cycle is completed at time index 17
[65 66]
 cycle is completed at time index 22
---------------------------------------------------------------
3


[(66, 16), (67, 17), (185, 18)]
[66]
 cycle is completed at time index 76
[ 66  67 183 184]
 cycle is completed at time index 81
[ 66  67 183 184 185]
 cycle is completed at time index 86
[66]
 cycle is completed at time index 148
[ 66  67 183 184]
 cycle is completed at time index 153
[ 66  67 183 184 185]
 cycle is completed at time index 158
---------------------------------------------------------------
